In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
import zipfile

# Unzip the training data
with zipfile.ZipFile("/kaggle/input/facial-keypoints-detection/training.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/training_data")

# Unzip the test data
with zipfile.ZipFile("/kaggle/input/facial-keypoints-detection/test.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/test_data")

# Read the training and test data into Pandas DataFrames
train = pd.read_csv('/kaggle/working/training_data/training.csv')
test = pd.read_csv('/kaggle/working/test_data/test.csv')
id = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/facial-keypoints-detection/training.zip'

In [2]:
display(train.head())

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


In [3]:
display(test.head())

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [4]:
display(id.head())

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN


In [5]:
train.isnull().sum()

left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [6]:
train.fillna(method='ffill', inplace=True)
train.isnull().sum()

/tmp/ipykernel_19/2125910151.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method='ffill', inplace=True)


left_eye_center_x            0
left_eye_center_y            0
right_eye_center_x           0
right_eye_center_y           0
left_eye_inner_corner_x      0
left_eye_inner_corner_y      0
left_eye_outer_corner_x      0
left_eye_outer_corner_y      0
right_eye_inner_corner_x     0
right_eye_inner_corner_y     0
right_eye_outer_corner_x     0
right_eye_outer_corner_y     0
left_eyebrow_inner_end_x     0
left_eyebrow_inner_end_y     0
left_eyebrow_outer_end_x     0
left_eyebrow_outer_end_y     0
right_eyebrow_inner_end_x    0
right_eyebrow_inner_end_y    0
right_eyebrow_outer_end_x    0
right_eyebrow_outer_end_y    0
nose_tip_x                   0
nose_tip_y                   0
mouth_left_corner_x          0
mouth_left_corner_y          0
mouth_right_corner_x         0
mouth_right_corner_y         0
mouth_center_top_lip_x       0
mouth_center_top_lip_y       0
mouth_center_bottom_lip_x    0
mouth_center_bottom_lip_y    0
Image                        0
dtype: int64

In [7]:
m,n = train.shape
print(m, '\t', n)

7049 	 31


In [8]:
test.shape

(1783, 2)

In [9]:
img = []
img_size = 96
for i in range(m):
    spliting = np.array(train['Image'][i].split(' '),dtype = 'float64')
    splitting = np.reshape(spliting,(img_size,img_size,1))
    splitting /= 255 
    img.append(splitting)
img = np.array(img)

In [10]:
X_train = img
train.drop('Image', axis = 1, inplace = True)
y_train = []

for i in range(len(train)):
    y = train.iloc[i,:].values
    y_train.append(y)
    
    
y_train= np.array(y_train, dtype = 'float')

In [11]:
model = Sequential([
    Conv2D(128, (3, 3),strides=1,activation='relu',padding = 'same', input_shape=(96, 96, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), strides=1,activation='relu',padding = 'same'),
    Dropout(0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), strides=1,activation='relu',padding = 'same'),
    Dropout(0.2),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), strides=1,activation='relu',padding = 'same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.1),
    Dense(256, activation='relu'),
    Dropout(0.1),
    Dense(128, activation='relu'),
    Dense(30)
])

In [12]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mae',metrics = ['accuracy'])

In [13]:
history = model.fit(X_train.reshape(-1, 96, 96, 1), y_train, epochs=2, batch_size=32,validation_split=0.2)

Epoch 1/2
177/177 [==============================] - 184s 1s/step - loss: 6.0454 - accuracy: 0.4293 - val_loss: 17.4484 - val_accuracy: 0.6872
Epoch 2/2
177/177 [==============================] - 181s 1s/step - loss: 4.0618 - accuracy: 0.5313 - val_loss: 12.3669 - val_accuracy: 0.5184


In [14]:
X_test = np.array([np.fromstring(x, dtype=int, sep=' ') for x in test['Image']])
X_test = X_test.reshape(-1, 96, 96, 1) / 255.0

y_test_pred = model.predict(X_test)

56/56 [==============================] - 14s 253ms/step


In [15]:
y_test_pred

array([[49.09755 , 27.50808 , 21.030262, ..., 55.39599 , 35.366455,
        58.661934],
       [48.59891 , 27.13123 , 20.830135, ..., 54.860237, 35.07116 ,
        58.107704],
       [50.63874 , 28.200516, 21.85893 , ..., 56.962673, 36.794632,
        60.24247 ],
       ...,
       [50.816498, 28.332445, 21.813023, ..., 57.094173, 37.148582,
        60.19443 ],
       [49.524216, 27.622957, 21.070314, ..., 55.75828 , 35.71714 ,
        58.860695],
       [49.97808 , 27.766216, 21.246826, ..., 56.265903, 35.981686,
        59.45563 ]], dtype=float32)

In [16]:
test_images = []
for i in range(len(test)):
    item = np.array(test['Image'][i].split(' '), dtype='float')
    item = np.reshape(item, (img_size, img_size, 1))
    item /= 255
    test_images.append(item)
    
test_images = np.array(test_images, dtype='float')

In [17]:
predict = model.predict(test_images)

56/56 [==============================] - 14s 249ms/step


In [18]:
feature_names = list(id['FeatureName'])
image_ids = list(id['ImageId']-1)
row_ids = list(id['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))

In [19]:
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(predict[x][y])
    

In [20]:
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('s.csv',index = False)